In [ ]:
!pip install azure-ai-projects azure-identity azure-ai-evaluation azure-ai-inference

以下の環境変数をセット。

- PROJECT_CONNECTION_STRING - The project connection string, as found in the overview page of your Azure AI Foundry project.
- MODEL_DEPLOYMENT_NAME - The deployment name of the model for AI-assisted evaluators, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.
- AZURE_OPENAI_ENDPOINT - Azure Open AI Endpoint to be used for evaluation.
- AZURE_OPENAI_API_KEY - Azure Open AI Key to be used for evaluation.
- AZURE_OPENAI_API_VERSION - Azure Open AI Api version to be used for evaluation.
- AZURE_SUBSCRIPTION_ID - Azure Subscription Id of Azure AI Project
- PROJECT_NAME - Azure AI Project Name
- RESOURCE_GROUP_NAME - Azure AI Project Resource Group Name
- AGENT_MODEL_DEPLOYMENT_NAME - The deployment name of the model for your Azure AI agent, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.

In [ ]:
from dotenv import load_dotenv

load_dotenv('./.env')

## Initialize Project Client

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import FunctionTool, ToolSet, AgentThreadCreationOptions
from user_functions import user_functions

project_client = AIProjectClient(
    credential=DefaultAzureCredential(),
    endpoint=os.environ["PROJECT_URL"],
)

AGENT_NAME = "Seattle Tourist Assistant"

# Add Tools to be used by Agent
functions = FunctionTool(user_functions)

toolset = ToolSet()
toolset.add(functions)

### AI Foundry Agent Service の作成

In [ ]:
agent = project_client.agents.create_agent(
    model=os.environ["AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME"],
    name=AGENT_NAME,
    instructions="You are helpful agent",
    toolset=toolset,
)
print(f"Created agent, agent ID: {agent.id}")

### Thread 作成、実行

In [ ]:
thread_run = project_client.agents.create_thread_and_process_run(
    agent_id=agent.id,
    thread=AgentThreadCreationOptions(
        messages=[
            {
                "role": "user",
                "content": "What are some good places to visit in Seattle?",
            }
        ]
    )
)
print(f"Thread run status: {thread_run.status}")

for message in project_client.agents.messages.list(thread_run.thread_id):
    print(f"Role: {message.role}, Content: {message.content[0].text.value}")

### 評価

In [ ]:
from azure.ai.evaluation import AIAgentConverter

# Initialize the converter that will be backed by the project.
converter = AIAgentConverter(project_client)

thread_id = thread_run.thread_id
run_id = thread_run.id
file_name = "evaluation_data.jsonl"

# Get a single agent run data
evaluation_data_single_run = converter.convert(thread_id=thread_id, run_id=run_id)
print(f"Single run evaluation data: {evaluation_data_single_run}")
evaluation_data = converter.prepare_evaluation_data(thread_ids=thread_id, filename=file_name)

### evaluator のセットアップ

In [ ]:
from azure.ai.evaluation import (
    ToolCallAccuracyEvaluator,
    AzureOpenAIModelConfiguration,
    IntentResolutionEvaluator,
    TaskAdherenceEvaluator,
)
from pprint import pprint

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AGENT_MODEL_DEPLOYMENT_NAME"],
)
# Needed to use content safety evaluators
azure_ai_project = {
    "subscription_id": os.environ["AZURE_SUBSCRIPTION_ID"],
    "project_name": os.environ["PROJECT_NAME"],
    "resource_group_name": os.environ["RESOURCE_GROUP_NAME"],
}

intent_resolution = IntentResolutionEvaluator(model_config=model_config)

tool_call_accuracy = ToolCallAccuracyEvaluator(model_config=model_config)

task_adherence = TaskAdherenceEvaluator(model_config=model_config)

In [ ]:
from azure.ai.evaluation import evaluate

response = evaluate(
    data=file_name,
    evaluators={
        "tool_call_accuracy": tool_call_accuracy,
        "intent_resolution": intent_resolution,
        "task_adherence": task_adherence,
    },
    azure_ai_project={
        "subscription_id": os.environ["AZURE_SUBSCRIPTION_ID"],
        "project_name": os.environ["PROJECT_NAME"],
        "resource_group_name": os.environ["RESOURCE_GROUP_NAME"],
    },
)
pprint(f'AI Foundary URL: {response.get("studio_url")}')

In [ ]:
from azure.ai.projects.models import AgentEvaluation, AgentEvaluationRequest


eval_job: AgentEvaluation = project_client.evaluations.create_agent_evaluation(
    evaluation=AgentEvaluationRequest(
        evaluators={
            
        },
        run_id=run_id
    )
)
print(f"Evaluation job created with ID: {eval_job.id}")

In [ ]:
import time

from azure.ai.projects.models import (
    AgentEvaluationRequest,
    EvaluatorIds,
    EvaluatorConfiguration,
    AgentEvaluationSamplingConfiguration,
    AgentEvaluationRedactionConfiguration,
)


thread = project_client.agents.threads.create()
print(f"Created thread, thread ID: {thread.id}")

message = project_client.agents.messages.create(
            thread_id=thread.id, role="user", content="Hello, tell me a joke"
        )
print(f"Created message, message ID: {message.id}")

run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id)

while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"Run status: {run.status}")

agent_evaluation_request = AgentEvaluationRequest(
    run_id=run.id,
    thread_id=thread.id,
    evaluators={
        "violence": EvaluatorConfiguration(
            id=EvaluatorIds.VIOLENCE,
        )
    },
    sampling_configuration=AgentEvaluationSamplingConfiguration(
        name="test",
        sampling_percent=100,
        max_request_rate=100,
    ),
    redaction_configuration=AgentEvaluationRedactionConfiguration(
        redact_score_properties=False,
    ),
    app_insights_connection_string=project_client.telemetry.get_connection_string(),
)

agent_evaluation_response = project_client.evaluations.create_agent_evaluation(
    evaluation=agent_evaluation_request
)

print(agent_evaluation_response)

In [ ]:
print(
    "Get an authenticated Azure OpenAI client for the parent AI Services resource, and perform a chat completion operation:"
)
with project_client.inference.get_azure_openai_client(api_version="2024-10-21") as client:

    response = client.chat.completions.create(
        model=os.environ["AGENT_MODEL_DEPLOYMENT_NAME"],
        messages=[
            {
                "role": "user",
                "content": "How many feet are in a mile?",
            },
        ],
    )

    print(response.choices[0].message.content)
